In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
image_size = (256, 256)
hydro_dir = r"C:\Users\nishi\EPICS\mmflood\mmflood\output\Hydro_Standardized"
dem_dir = r"C:\Users\nishi\EPICS\mmflood\mmflood\output\DEM_Standardized"
s1_raw_dir = r"C:\Users\nishi\EPICS\mmflood\mmflood\output\S1_RGB_Img_Standardized"
mask_dir = r"C:\Users\nishi\EPICS\mmflood\mmflood\output\Mask_Standardized"

In [3]:
hydro_files = sorted(os.listdir(hydro_dir))
dem_files = sorted(os.listdir(dem_dir))
s1_raw_files = sorted(os.listdir(s1_raw_dir))
mask_files = sorted(os.listdir(mask_dir))

common_files = set(hydro_files) & set(dem_files) & set(s1_raw_files) & set(mask_files)
common_files = sorted(list(common_files))

In [4]:
def load_common_images(image_paths, common_files):
    images = []
    for file in common_files:
        img_path = os.path.join(image_paths, file)
        img = load_img(img_path, target_size=image_size)
        img = img_to_array(img) / 255.0
        images.append(img)
    return np.array(images)

In [5]:
hydro_data = load_common_images(hydro_dir, common_files)
dem_data = load_common_images(dem_dir, common_files)
s1_raw_data = load_common_images(s1_raw_dir, common_files)
mask_data = load_common_images(mask_dir, common_files)

In [6]:
mask_data = (mask_data > 0.5).astype(np.uint8)

In [7]:
X = np.concatenate([hydro_data, dem_data, s1_raw_data], axis=-1)
y = mask_data

In [8]:
#X = X.astype('float16')

In [9]:
print(f"Hydro Data Shape: {hydro_data.shape}")
print(f"DEM Data Shape: {dem_data.shape}")
print(f"S1 Raw Data Shape: {s1_raw_data.shape}")
print(f"Mask Data Shape: {mask_data.shape}")
print(f"Final Input Shape (X): {X.shape}")

Hydro Data Shape: (1012, 256, 256, 3)
DEM Data Shape: (1012, 256, 256, 3)
S1 Raw Data Shape: (1012, 256, 256, 3)
Mask Data Shape: (1012, 256, 256, 3)
Final Input Shape (X): (1012, 256, 256, 9)


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=176)
print(f"Training Data Shape: {X_train.shape}, Validation Data Shape: {X_val.shape}")

Training Data Shape: (809, 256, 256, 9), Validation Data Shape: (203, 256, 256, 9)


In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Input
from tensorflow.keras.models import Model

In [18]:
def unet_model(input_size=(256, 256, 9)):  
    inputs = Input(input_size)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    up1 = UpSampling2D(size=(2, 2))(conv3)
    merge1 = concatenate([conv2, up1], axis=-1)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge1)
    up2 = UpSampling2D(size=(2, 2))(conv4)
    merge2 = concatenate([conv1, up2], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge2)
    output = Conv2D(1, (1, 1), activation='sigmoid')(conv5)  # Binary mask
    model = Model(inputs, output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
model = unet_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 9)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_6 (Conv2D)             │ (None, 256, 256, 64)      │           5,248 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 128, 128, 64)      │               0 │ conv2d_6[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_7 (Conv2D)             │ (None, 128, 128, 128)     │          73,856 │ max_pooling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_3               │ (None, 64, 64, 128)       │               0 │ conv2d_7[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_8 (Conv2D)             │ (None, 64, 64, 256)       │         295,168 │ max_pooling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_2               │ (None, 128, 128, 256)     │               0 │ conv2d_8[0][0]             │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_2 (Concatenate)   │ (None, 128, 128, 384)     │               0 │ conv2d_7[0][0],            │
│                               │                           │                 │ up_sampling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_9 (Conv2D)             │ (None, 128, 128, 128)     │         442,496 │ concatenate_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_3               │ (None, 256, 256, 128)     │               0 │ conv2d_9[0][0]             │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_3 (Concatenate)   │ (None, 256, 256, 192)     │               0 │ conv2d_6[0][0],            │
│                               │                           │                 │ up_sampling2d_3[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_10 (Conv2D)            │ (None, 256, 256, 64)      │         110,656 │ concatenate_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_11 (Conv2D)            │ (None, 256, 256, 1)       │              65 │ conv2d_10[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 927,489 (3.54 MB)

 Trainable params: 927,489 (3.54 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
y_train = y_train[..., :1]  
y_val = y_val[..., :1]

In [20]:
def train_generator(X, Y, batch_size):
    while True:  
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            Y_batch = Y[i:i+batch_size]
            yield X_batch, Y_batch  

In [ ]:
history = model.fit(train_generator(X_train, y_train, batch_size=batch_size), validation_data=(X_val, y_val),
                    steps_per_epoch=len(X_train) // batch_size, epochs=epochs) 
# 101 passes and 50 epochs

In [23]:
history = model.fit(train_generator(X_train, y_train, batch_size=batch_size), validation_data=(X_val, y_val),
                    steps_per_epoch=len(X_train) // batch_size, epochs=10)

Epoch 1/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 380s 8s/step - accuracy: 0.9501 - loss: 0.1561 - val_accuracy: 0.9516 - val_loss: 0.1555
Epoch 2/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 371s 7s/step - accuracy: 0.9512 - loss: 0.1552 - val_accuracy: 0.9532 - val_loss: 0.1472
Epoch 3/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 400s 8s/step - accuracy: 0.9539 - loss: 0.1483 - val_accuracy: 0.9540 - val_loss: 0.1457
Epoch 4/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 373s 7s/step - accuracy: 0.9556 - loss: 0.1440 - val_accuracy: 0.9581 - val_loss: 0.1346
Epoch 5/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 366s 7s/step - accuracy: 0.9584 - loss: 0.1339 - val_accuracy: 0.9585 - val_loss: 0.1348
Epoch 6/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 392s 8s/step - accuracy: 0.9593 - loss: 0.1320 - val_accuracy: 0.9590 - val_loss: 0.1324
Epoch 7/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 422s 8s/step - accuracy: 0.9613 - loss: 0.1269 - val_accuracy: 0.9609 - val_loss: 0.1281
Epoch 8/8
50/50 ━━━━━━━━━━━━━━━━━━━━ 375s 7s/step - accuracy: 0.9603 - loss: 0.1283 - val_accuracy: 0.9610 - val_loss:

In [24]:
model.save("mera_Unet_flood_model.h5")

In [26]:
model.save_weights("mera_Unet_model_weights.weights.h5")

In [27]:
model.save("mera_Unet_flood_model.keras")  # full save